In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime
import time
from tqdm import tqdm
import sys
import glob

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

# Submited OOF CV

In [77]:
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())
pred_col = 'prediction'
lb3664 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/0215_083_stack_submit_OUT29-7593_CV3-61266_LB3-664.gz')[[key, pred_col]].set_index(key)
lb3667 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/0213_074_submit_base-stack-lgb8_rm_out-lgb1_no_out-0_clf-thres0.0082_CV3.6088898700520273_LB3.667.gz')[[key, pred_col]].set_index(key)
base['PP_LB3-664_CV3-61266_OutlierFit29-7593_clf-thres0-01'] = lb3664[pred_col]
base['PP_LB3-667_CV3-61100_OutlierFit0_clf-thres0-082'] = lb3667[pred_col]
utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0219_go_OOF_CV_Submited_2MODEL_STACK_With_Outlier_or_Without_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_Submited_2MODEL_STACK_With_Outlier_or_Without_{len(base.columns)-1}set.csv')
base.head()

100%|██████████| 3/3 [00:00<00:00, 57.30it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


,target,PP_LB3-664_CV3-61266_OutlierFit29-7593_clf-thres0-01,PP_LB3-667_CV3-61100_OutlierFit0_clf-thres0-082
card_id,,,
C_ID_92a2005557,-0.820283,-0.343964,-0.335063
C_ID_3d0044924f,0.392913,0.226026,0.374489
C_ID_d639edf6cd,0.688056,0.750429,0.824915
C_ID_186d6a6901,0.142495,0.193548,0.130264
C_ID_cdbd2c0db2,-0.159749,-0.258822,-0.190934


# Stack前のRremove Outlier Single Model OOF

In [78]:
#========================================================================
# Stack前のRremove Outlier Single Model OOF
#========================================================================
    
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())

rm_out_list = glob.glob('../ensemble/rm_outlier_ensemble/*lgb*199710*.gz')

for path in rm_out_list:
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    if filename.count('CV8'):
        continue
    cv = re.search(r'CV([^/.]*)_LB', filename).group(1)
    lb = re.search(r'LB([^/.]*)', filename).group(1)
    
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
        
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0

    try:
        valid_type = re.search(r'valid-([^/%d]*)_E', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        elif filename.count('term'):
            valid_type = 'term'
        else:
            valid_type = '0'
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
    if filename.count('ridge'):
        col_name = f'ridge_lgb_stack_{col_name}'
    
    print(col_name)
    base[col_name] = tmp[pred_col]
utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0219_go_OOF_CV_Without_Outlier_GROUP_Single_LGB_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_Without_Outlier_GROUP_Single_LGB_{len(base.columns)-1}set.csv')
base.head()    

100%|██████████| 3/3 [00:00<00:00, 54.82it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


CV1-5540_LB_set1_feat340_valid-0
CV1-5520_LB_set3_feat290_valid-term
CV1-5530_LB_set3_feat290_valid-pm
CV1-5500_LB_set0_feat235_valid-ods
CV1-5500_LB_set0_feat235_valid-ods
CV1-5500_LB_set0_feat235_valid-pm
CV1-5540_LB_set99_feat340_valid-pm
CV1-5520_LB_set3_feat290_valid-pm
CV1-5520_LB_set3_feat290_valid-0
CV1-5500_LB_set0_feat235_valid-pm
CV1-5530_LB_set3_feat290_valid-ods
CV1-5552_LB_set0_feat235_valid-ods_obj-huber
CV1-5494_LB_set0_feat235_valid-0
CV1-5520_LB_set0_feat235_valid-ods_obj-fair
CV1-5497_LB_set0_feat235_valid-term
CV1-5530_LB_set2_feat320_valid-ods
CV1-5530_LB_set2_feat320_valid-ods


,target,CV1-5540_LB_set1_feat340_valid-0,CV1-5520_LB_set3_feat290_valid-term,CV1-5530_LB_set3_feat290_valid-pm,CV1-5500_LB_set0_feat235_valid-ods,CV1-5500_LB_set0_feat235_valid-pm,CV1-5540_LB_set99_feat340_valid-pm,CV1-5520_LB_set3_feat290_valid-pm,CV1-5520_LB_set3_feat290_valid-0,CV1-5530_LB_set3_feat290_valid-ods,CV1-5552_LB_set0_feat235_valid-ods_obj-huber,CV1-5494_LB_set0_feat235_valid-0,CV1-5520_LB_set0_feat235_valid-ods_obj-fair,CV1-5497_LB_set0_feat235_valid-term,CV1-5530_LB_set2_feat320_valid-ods
card_id,,,,,,,,,,,,,,,
C_ID_92a2005557,-0.820283,-0.205142,-0.298513,-0.272369,-0.330387,-0.322292,-0.265328,-0.305220,-0.263605,-0.269807,-0.325296,-0.308104,-0.311772,-0.351748,-0.341538
C_ID_3d0044924f,0.392913,0.260930,0.370687,0.665407,0.499904,0.230232,0.376686,0.446242,0.365161,0.535681,0.327556,0.281397,0.303839,0.221988,0.050486
C_ID_d639edf6cd,0.688056,0.821572,0.745334,0.759594,0.837948,0.846836,0.716802,0.751869,0.797271,0.781336,0.713505,0.837285,0.714139,0.741589,0.632040
C_ID_186d6a6901,0.142495,0.212679,0.141234,0.162507,0.119828,0.196142,0.245714,0.130344,0.147096,0.128310,0.303504,0.061090,0.224406,0.218610,0.254490
C_ID_cdbd2c0db2,-0.159749,-0.164458,-0.200087,-0.194075,-0.212314,-0.250468,-0.124312,-0.175082,-0.172849,-0.159000,-0.326135,-0.217998,-0.271558,-0.230634,-0.256391


# RidgeによるLGBのStacking OOF. Exclude Outlier

In [79]:
#========================================================================
# RidgeによるLGBのStacking OOF. Exclude Outlier
#========================================================================
    
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())

# rm_out_list = glob.glob('../stack/*ridge*rm_out*.gz')
rm_out_list = ['../stack/0215_083_stack_ridge_set-rm_out_lgb20_NN0_other0_OUT0_CV1.5464357357745486_LB.gz']

for path in rm_out_list:
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    if filename.count('CV8'):
        continue
    cv = re.search(r'CV([^/.]*)_LB', filename).group(1)
    lb = re.search(r'LB([^/.]*)', filename).group(1)
    
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
        
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0

    try:
        valid_type = re.search(r'valid-([^/%d]*)_E', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        elif filename.count('term'):
            valid_type = 'term'
        else:
            valid_type = '0'
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
    if filename.count('ridge'):
        col_name = f'ridge_lgb_stack_{col_name}'
    
    print(col_name)
    base[col_name] = tmp[pred_col]
utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0219_go_OOF_CV_Without_Outlier_GROUP_Ridge_Stack_Some_LGB_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_Without_Outlier_GROUP_Ridge_Stack_Some_LGB_{len(base.columns)-1}set.csv')
base.head()    

100%|██████████| 3/3 [00:00<00:00, 53.09it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


ridge_lgb_stack_CV1-5464_LB_set99_feat0_valid-0


,target,ridge_lgb_stack_CV1-5464_LB_set99_feat0_valid-0
card_id,,
C_ID_92a2005557,-0.820283,-0.343964
C_ID_3d0044924f,0.392913,0.226026
C_ID_d639edf6cd,0.688056,0.750429
C_ID_186d6a6901,0.142495,0.193548
C_ID_cdbd2c0db2,-0.159749,-0.258822


# RidgeによるLGBのStacking OOF. All GROUP Dataset

In [80]:
#========================================================================
# RidgeによるLGBのStacking OOF. All GROUP Dataset
#========================================================================
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())

# oof_path_list = glob.glob('../ensemble/good_submit_ensemble/*.gz')
oof_path_list = glob.glob('../stack/*ridge*set-all*.gz')

for path in oof_path_list:
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    cv = re.search(r'CV([^/.]*)_LB', filename).group(1)
    lb = re.search(r'LB([^/.]*)', filename).group(1)
    
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
        
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0

    try:
        valid_type = re.search(r'valid-([^/%d]*)_', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        else:
            valid_type = '0'
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
    if filename.count('ridge'):
        col_name = f'ridge_lgb_stack_{col_name}'
    
    print(col_name)
    base[col_name] = tmp[pred_col]
utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0219_go_OOF_CV_Ridge_Stack_Some_LGB_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_Ridge_Stack_Some_LGB_{len(base.columns)-1}set.csv')
base.head()

100%|██████████| 3/3 [00:00<00:00, 54.64it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


ridge_lgb_stack_CV3-6170_LB_set99_feat0_valid-pmo
ridge_lgb_stack_CV3-6188_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6185_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6168_LB_set99_feat0_valid-ods
ridge_lgb_stack_CV3-6166_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6192_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6186_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6162_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6169_LB_set99_feat0_valid-pm
ridge_lgb_stack_CV3-6168_LB_set99_feat0_valid-ods
ridge_lgb_stack_CV3-6170_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6169_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6170_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6173_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6185_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6182_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6182_LB_set99_feat0_valid-pmo
ridge_lgb_stack_CV3-6190_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6183_LB_set99_feat0_valid-0
ridge_lgb_stack_CV3-6170_LB_set99_feat0_valid-pm


,target,ridge_lgb_stack_CV3-6170_LB_set99_feat0_valid-pmo,ridge_lgb_stack_CV3-6188_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6185_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6168_LB_set99_feat0_valid-ods,ridge_lgb_stack_CV3-6166_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6192_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6186_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6162_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6169_LB_set99_feat0_valid-pm,ridge_lgb_stack_CV3-6170_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6169_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6173_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6182_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6182_LB_set99_feat0_valid-pmo,ridge_lgb_stack_CV3-6190_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6183_LB_set99_feat0_valid-0,ridge_lgb_stack_CV3-6170_LB_set99_feat0_valid-pm
card_id,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-0.820283,-0.295880,-0.315972,-0.336581,-0.290303,-0.291631,-0.317954,-0.341483,-0.286590,-0.315837,-0.351792,-0.330954,-0.337321,-0.347561,-0.315029,-0.341280,-0.338368,-0.303976
C_ID_3d0044924f,0.392913,-0.477898,-0.359790,-0.397216,-0.462763,-0.490546,-0.379105,-0.411978,-0.472667,-0.575479,-0.456843,-0.385938,-0.434009,-0.408186,-0.450728,-0.415895,-0.378602,-0.462785
C_ID_d639edf6cd,0.688056,0.687242,0.726963,0.728027,0.710084,0.679473,0.715908,0.727070,0.676653,0.711340,0.731350,0.727698,0.725854,0.728502,0.746781,0.718447,0.728075,0.700509
C_ID_186d6a6901,0.142495,0.182706,0.207114,0.208479,0.166960,0.179082,0.205325,0.192469,0.166292,0.179746,0.178674,0.203129,0.200703,0.195306,0.164986,0.203448,0.195535,0.171668
C_ID_cdbd2c0db2,-0.159749,-0.075056,-0.030583,-0.052372,-0.106814,-0.082665,-0.033592,-0.061768,-0.088617,-0.085113,-0.086179,-0.037800,-0.036709,-0.063038,-0.099842,-0.061033,-0.054785,-0.099401


# Stacking前のLGB OOF CV Set

In [18]:
#========================================================================
# Stacking前のLGB OOF CV Set
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())

lgb_path_list = glob.glob('../model/LB3664_set/*.gz') + glob.glob('../ensemble/lgb_ensemble/*.gz')
# lgb_path_list += glob.glob('../stack/10seed/*.gz')

for path in lgb_path_list:
    
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    
    if filename.count('NN') or filename.count('blend'):
        continue
    
    cv = re.search(r'CV([^/.]*)_LB', filename).group(1)
    lb = re.search(r'LB([^/.]*)', filename).group(1)
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0
    try:
        valid_type = re.search(r'valid-([^/%d]*)_', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        else:
            valid_type = '0'
        
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:6]}_LB{lb}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
    print(col_name)
    base[col_name] = tmp[pred_col]

# utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0220_go_OOF_CV_single_LGB_{len(base.columns)-1}set.csv')
utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_various_model/0220_go_OOF_CV_single_LGB_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_single_LGB_{len(base.columns)-1}set.csv')
base.head()
#========================================================================

100%|██████████| 3/3 [00:00<00:00, 58.77it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


CV3-6430_LB_set99_feat352_valid-0
CV3-6335_LB_set99_feat173_valid-0
CV3-6216_LB_set99_feat296_valid-0
CV3-6213_LB_set0_feat235_valid-pmo
CV3-6297_LB_set99_feat338_valid-0
CV3-6402_LB_set2_feat320_valid-pmo
CV3-2381_LB_set99_feat272_valid-0
CV3-2400_LB_set99_feat261_valid-0
CV3-6431_LB_set99_feat303_valid-0
CV3-6514_LB_set99_feat501_valid-0
CV3-6435_LB_set1_feat340_valid-pm
CV3-6246_LB_set99_feat295_valid-0
CV3-6236_LB_set99_feat297_valid-0
CV3-6333_LB_set99_feat0_valid-0
CV3-6536_LB_set99_feat306_valid-0
CV3-6430_LB_set99_feat352_valid-0
CV3-6239_LB_set99_feat219_valid-0
CV3-6208_LB_set99_feat243_valid-0
CV3-6241_LB_set99_feat204_valid-0
CV3-6297_LB_set99_feat191_valid-0
CV3-6182_LB_set0_feat277_valid-ods
CV3-6233_LB_set99_featstack_lgb_lr0-01_219_valid-0
CV3-7554_LB_set99_feat235_valid-ods_obj-fair
CV3-6297_LB_set99_feat156_valid-0
CV3-6434_LB_set99_feat340_valid-0
CV3-6241_LB_set99_feat222_valid-0
CV3-6186_LB_set0_feat288_valid-ods
CV3-6208_LB_set99_feat260_valid-0
CV3-6181_LB_set0_f

,target,CV3-6430_LB_set99_feat352_valid-0,CV3-6335_LB_set99_feat173_valid-0,CV3-6216_LB_set99_feat296_valid-0,CV3-6213_LB_set0_feat235_valid-pmo,CV3-6297_LB_set99_feat338_valid-0,CV3-6402_LB_set2_feat320_valid-pmo,CV3-2381_LB_set99_feat272_valid-0,CV3-2400_LB_set99_feat261_valid-0,CV3-6431_LB_set99_feat303_valid-0,CV3-6514_LB_set99_feat501_valid-0,CV3-6435_LB_set1_feat340_valid-pm,CV3-6246_LB_set99_feat295_valid-0,CV3-6236_LB_set99_feat297_valid-0,CV3-6333_LB_set99_feat0_valid-0,CV3-6536_LB_set99_feat306_valid-0,CV3-6239_LB_set99_feat219_valid-0,CV3-6208_LB_set99_feat243_valid-0,CV3-6241_LB_set99_feat204_valid-0,CV3-6297_LB_set99_feat191_valid-0,CV3-6182_LB_set0_feat277_valid-ods,CV3-6233_LB_set99_featstack_lgb_lr0-01_219_valid-0,CV3-7554_LB_set99_feat235_valid-ods_obj-fair,CV3-6297_LB_set99_feat156_valid-0,CV3-6434_LB_set99_feat340_valid-0,CV3-6241_LB_set99_feat222_valid-0,CV3-6186_LB_set0_feat288_valid-ods,CV3-6208_LB_set99_feat260_valid-0,CV3-6181_LB_set0_feat278_valid-ods,CV3-6212_LB_set99_feat245_valid-0,CV3-6238_LB_set99_feat238_valid-0,CV3-6330_LB_set99_feat125_valid-0,CV3-6223_LB_set99_feat237_valid-0,CV3-6220_LB_set99_feat238_valid-0,CV3-6206_LB_set99_feat252_valid-0,CV3-6217_LB_set99_feat238_valid-0,CV3-6241_LB_set0_feat236_valid-ods,CV3-6314_LB_set99_feat126_valid-0,CV3-6208_LB_set99_feat258_valid-0,CV3-6341_LB_set99_feat138_valid-0,CV3-6416_LB_set2_feat320_valid-pm,CV3-6335_LB_set99_feat195_stack_lgb_lr0-01_271_valid-0,CV3-6280_LB_set99_feat188_valid-0,CV3-7859_LB_set99_feat235_valid-ods_obj-huber,CV3-6203_LB_set99_feat250_valid-0,CV3-6319_LB_set99_feat124_valid-0,CV3-6221_LB_set99_feat264_valid-0,CV3-6178_LB_set4_feat272_valid-ods,CV3-6229_LB_set99_feat223_valid-0,CV3-6439_LB_set99_feat303_valid-0,CV3-6216_LB_set99_feat236_valid-0,CV3-6327_LB_set99_feat133_valid-0,CV3-6274_LB_set99_feat191_valid-0,CV3-6237_LB_set99_feat207_valid-0,CV3-6228_LB_set99_feat225_valid-0,CV3-6178_LB_set0_feat266_valid-ods,CV3-6453_LB_set1_feat340_valid-pm,CV3-6189_LB_set0_feat278_valid-ods,CV3-6208_LB_set99_feat247_valid-0,CV3-6183_LB_set0_feat305_valid-ods,CV3-6296_LB_set99_feat147_valid-0,CV3-6232_LB_set99_feat220_valid-0,CV3-6192_LB_set4_feat322_valid-ods,CV3-6179_LB_set0_feat274_valid-ods,CV3-6232_LB_set99_feat214_valid-0,CV3-6413_LB_set3_feat290_valid-ods,CV3-6238_LB_set99_feat202_valid-0,CV3-6248_LB_set0_feat235_valid-pm,CV3-6218_LB_set99_feat235_valid-0,CV3-9313_LB_set99_feat235_valid-0,CV3-6394_LB_set99_feat320_valid-0,CV3-6337_LB_set99_feat231_stack_lgb_lr0-01_122_valid-0,CV3-6283_LB_set99_feat169_valid-0,CV3-6180_LB_set0_feat266_valid-ods,CV3-6257_LB_set0_feat236_valid-ods,CV3-6180_LB_set4_feat273_valid-ods,CV3-6242_LB_set99_feat274_valid-0,CV3-6284_LB_set99_feat156_valid-0,CV3-6209_LB_set99_feat245_valid-0,CV3-6314_LB_set99_feat131_valid-0,CV3-8272_LB_set0_feat235_valid-ods_obj-mape,CV3-6290_LB_set99_feat156_valid-0,CV3-6324_LB_set99_feat134_valid-0,CV3-6178_LB_set0_feat259_valid-ods,CV3-6221_LB_set0_feat235_valid-pmo,CV3-6203_LB_set99_feat251_valid-0,CV3-6237_LB_set99_feat219_valid-0,CV3-6126_LB3-664_set99_feat0_valid-0,CV3-6315_LB_set99_feat128_valid-0,CV3-6232_LB_set99_feat221_valid-0,CV3-6174_LB_set0_feat288_valid-ods,CV3-6219_LB_set99_feat240_valid-0,CV3-6192_LB_set0_feat291_valid-ods,CV3-6467_LB_set3_feat290_valid-pmo,CV3-6227_LB_set99_feat231_valid-0,CV3-6238_LB_set99_feat276_valid-0,CV3-6439_LB_set99_feat290_valid-0,CV3-6232_LB_set99_feat234_valid-0,CV3-6282_LB_set99_feat166_valid-0,CV3-6221_LB_set99_feat233_valid-0,CV3-6412_LB_set2_feat320_valid-ods,CV3-6284_LB_set99_feat185_valid-0,CV3-6181_LB_set0_feat264_valid-ods,CV3-6236_LB_set99_feat228_valid-0,CV3-6205_LB_set99_feat260_valid-0,CV3-6288_LB_set99_feat175_valid-0,CV3-6216_LB_set99_feat238_valid-0,CV3-6181_LB_set0_feat267_valid-ods,CV3-6286_LB_set99_feat188_valid-0,CV3-6437_LB_set3_feat290_valid-out,CV3-6341_LB_set99_feat192_stack_lgb_lr0-01_304_valid-0,CV3-6401_LB_set1_feat340_valid-ods,CV3-6278_LB_set99_feat186_valid-0,CV3-6468_LB_set1_feat340_valid-pmo,CV3-6211_

## blend前の Clf OOF CV Set

In [6]:
#========================================================================
# Classifier
base = utils.read_df_pkl('../input/base_term*0*')[[key, target]].set_index(key)
display(base.head())

lgb_path_list = glob.glob('../stack/0207*binary*.gz')

for path in lgb_path_list:
    
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    
    if filename.count('NN') or filename.count('blend'):
        continue
    
    cv = re.search(r'CV([^/.]*)_', filename).group(1)
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0
    try:
        valid_type = re.search(r'valid-([^/%d]*)_', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        else:
            valid_type = '0'
        
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:6]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:6]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:6]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:6]}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
#     print(col_name)
    base[col_name] = tmp[pred_col]

utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0220_go_OOF_CV_Outlier_Clf_single_LGB_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_single_LGB_{len(base.columns)-1}set.csv')
base.head()
#========================================================================

100%|██████████| 3/3 [00:00<00:00, 42.47it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


,target,CV0-9099_set99_feat0_valid-0,CV0-9084_set99_feat0_valid-0
card_id,,,
C_ID_92a2005557,-0.820283,0.000426,0.000601
C_ID_3d0044924f,0.392913,0.008079,0.005600
C_ID_d639edf6cd,0.688056,0.004211,0.002847
C_ID_186d6a6901,0.142495,0.000789,0.000869
C_ID_cdbd2c0db2,-0.159749,0.000259,0.000187


### ~201901のLGB

In [20]:
#========================================================================
# Stacking前のLGB OOF CV Set
base = utils.read_df_pkl('../input/base_no_out_clf.gz')[[key, target]].set_index(key)
display(base.head())

lgb_path_list = glob.glob('../stack/dir_01/*.gz')

for path in lgb_path_list:
    
    filename = re.search(r'/([^/.]*).gz', path.replace('.', '-')).group(1)
    if filename.count('CV3-35'):
        print(filename)
    
    if filename.count('NN') or filename.count('blend'):
        continue
    
    cv = re.search(r'CV([^/.]*)_', filename).group(1)
    try:
        eset = re.search(r'ESET([^/.])_', filename).group(1)
    except AttributeError:
        eset = 99
    try:
        feat_num = re.search(r'01_([^/%d]*)feats', filename).group(1)
    except AttributeError:
        feat_num = 0
    try:
        valid_type = re.search(r'valid-([^/%d]*)_', filename).group(1)
    except AttributeError:
        if filename.count('ods'):
            valid_type = 'ods'
        elif filename.count('pmo'):
            valid_type = 'pmo'
        elif filename.count('pm'):
            valid_type = 'pm'
        else:
            valid_type = '0'
        
    
    tmp = utils.read_pkl_gzip(path)
    if 'pred_mean' in tmp.columns:
        tmp = tmp[[key, 'pred_mean']].set_index(key)
        pred_col = 'pred_mean'
    else:
        tmp = tmp[[key, 'prediction']].set_index(key)
        pred_col = 'prediction'
        
    if filename.count('huber'):
        col_name = f'CV{cv[:7]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-huber'
    elif filename.count('fair'):
        col_name = f'CV{cv[:7]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-fair'
    elif filename.count('mape'):
        col_name = f'CV{cv[:7]}_set{eset}_feat{feat_num}_valid-{valid_type}_obj-mape'
    else:
        col_name = f'CV{cv[:7]}_set{eset}_feat{feat_num}_valid-{valid_type}'
        
    print(col_name)
    base[col_name] = tmp[pred_col]

utils.to_pkl_gzip(obj=base.reset_index(), path=f'../kaggle_oof_set/0220_go_OOF_CV_single_LGB_Various_Validation_Set_Train_{len(base.columns)-1}set.csv')
# base.to_csv(f'../kaggle_oof_set/0219_go_OOF_CV_single_LGB_{len(base.columns)-1}set.csv')
base.head()
# ========================================================================

100%|██████████| 1/1 [00:00<00:00,  3.98it/s]


,target
card_id,
C_ID_92a2005557,-0.820283
C_ID_3d0044924f,0.392913
C_ID_d639edf6cd,0.688056
C_ID_186d6a6901,0.142495
C_ID_cdbd2c0db2,-0.159749


CV3-07015_set99_feat172_valid-pmo
CV5-24805_set99_feat172_valid-pm
CV3-31028_set99_feat172_valid-pmo
CV4-28736_set99_feat172_valid-pm
CV3-64902_set99_feat200_valid-0
CV8-15579_set99_feat172_valid-pmo
CV3-64871_set99_feat193_valid-0
CV3-64429_set99_feat172_valid-pm
CV3-64977_set99_feat186_valid-0
CV6-66725_set99_feat172_valid-pmo
CV3-64005_set99_feat172_valid-0
CV5-44443_set99_feat172_valid-pmo
CV7-74755_set99_feat172_valid-pm
CV3-64942_set99_feat188_valid-0
CV3-63678_set99_feat172_valid-0
CV3-38100_set99_feat172_valid-pmo
CV4-80832_set99_feat172_valid-pmo
CV7-65202_set99_feat172_valid-pm
CV5-58694_set99_feat172_valid-pm
CV4-43351_set99_feat172_valid-pmo
CV3-56606_set99_feat172_valid-pmo
CV3-65053_set99_feat177_valid-0
CV4-66190_set99_feat172_valid-pm
CV3-42651_set99_feat172_valid-pm
CV4-91818_set99_feat172_valid-ods
CV3-64031_set99_feat172_valid-0
CV3-25264_set99_feat172_valid-pm
CV3-73040_set99_feat172_valid-pmo
CV3-64887_set99_feat159_valid-0
CV4-41248_set99_feat172_valid-pmo
CV4-907

,target,CV3-07015_set99_feat172_valid-pmo,CV5-24805_set99_feat172_valid-pm,CV3-31028_set99_feat172_valid-pmo,CV4-28736_set99_feat172_valid-pm,CV3-64902_set99_feat200_valid-0,CV8-15579_set99_feat172_valid-pmo,CV3-64871_set99_feat193_valid-0,CV3-64429_set99_feat172_valid-pm,CV3-64977_set99_feat186_valid-0,CV6-66725_set99_feat172_valid-pmo,CV3-64005_set99_feat172_valid-0,CV5-44443_set99_feat172_valid-pmo,CV7-74755_set99_feat172_valid-pm,CV3-64942_set99_feat188_valid-0,CV3-63678_set99_feat172_valid-0,CV3-38100_set99_feat172_valid-pmo,CV4-80832_set99_feat172_valid-pmo,CV7-65202_set99_feat172_valid-pm,CV5-58694_set99_feat172_valid-pm,CV4-43351_set99_feat172_valid-pmo,CV3-56606_set99_feat172_valid-pmo,CV3-65053_set99_feat177_valid-0,CV4-66190_set99_feat172_valid-pm,CV3-42651_set99_feat172_valid-pm,CV4-91818_set99_feat172_valid-ods,CV3-64031_set99_feat172_valid-0,CV3-25264_set99_feat172_valid-pm,CV3-73040_set99_feat172_valid-pmo,CV3-64887_set99_feat159_valid-0,CV4-41248_set99_feat172_valid-pmo,CV4-90772_set99_feat172_valid-pm,CV3-64784_set99_feat209_valid-0,CV3-77163_set99_feat172_valid-pmo,CV3-80555_set99_feat172_valid-pmo,CV3-87660_set99_feat172_valid-pm,CV3-64972_set99_feat175_valid-0,CV4-12505_set99_feat172_valid-pm,CV4-08795_set99_feat172_valid-pm,CV3-64849_set99_feat182_valid-0,CV4-16515_set99_feat172_valid-pmo,CV7-91790_set99_feat172_valid-pmo,CV4-64219_set99_feat172_valid-pm,CV5-62703_set99_feat172_valid-pm,CV4-39195_set99_feat172_valid-pmo,CV3-63924_set99_feat172_valid-0,CV4-64215_set99_feat172_valid-pm,CV3-64904_set99_feat200_valid-0,CV3-42480_set99_feat172_valid-pm,CV3-22993_set99_feat172_valid-pmo,CV3-92925_set99_feat172_valid-pm,CV4-41717_set99_feat172_valid-pmo,CV7-52653_set99_feat172_valid-pmo,CV3-64951_set99_feat204_valid-0,CV3-64982_set99_feat175_valid-0,CV3-64926_set99_feat193_valid-0,CV5-41288_set99_feat172_valid-pm,CV3-54052_set99_feat172_valid-pmo,CV5-59204_set99_feat172_valid-pm,CV4-37218_set99_feat172_valid-pm,CV5-45854_set99_feat172_valid-pm,CV3-14081_set99_feat172_valid-pmo,CV3-89331_set99_feat172_valid-pm,CV3-65019_set99_feat180_valid-0,CV5-37798_set99_feat172_valid-pm,CV3-75348_set99_feat172_valid-pmo,CV5-52635_set99_feat172_valid-pmo,CV5-79755_set99_feat172_valid-pm,CV4-14460_set99_feat172_valid-pmo,CV3-64798_set99_feat190_valid-0,CV3-64642_set99_feat179_valid-0,CV4-47120_set99_feat172_valid-pm,CV4-11991_set99_feat172_valid-pmo,CV3-66376_set99_feat169_valid-0,CV5-92955_set99_feat172_valid-pm,CV3-20537_set99_feat172_valid-pm,CV4-66412_set99_feat172_valid-pm,CV3-64961_set99_feat169_valid-0,CV7-66402_set99_feat172_valid-pmo,CV3-64330_set99_feat180_valid-0,CV4-69120_set99_feat172_valid-pm,CV3-65623_set99_feat172_valid-pmo,CV3-96563_set99_feat172_valid-pm,CV3-64904_set99_feat205_valid-0,CV5-50175_set99_feat172_valid-pm,CV4-55551_set99_feat172_valid-pmo,CV3-27062_set99_feat172_valid-pmo,CV3-83461_set99_feat172_valid-pm,CV6-65360_set99_feat172_valid-pmo,CV4-91596_set99_feat172_valid-ods,CV3-13102_set99_feat172_valid-pm,CV5-45840_set99_feat172_valid-pmo,CV3-64114_set99_feat180_valid-0,CV3-64817_set99_feat184_valid-0,CV3-64961_set99_feat197_valid-0,CV3-65036_set99_feat192_valid-0,CV3-64091_set99_feat180_valid-0,CV4-57669_set99_feat172_valid-pm,CV3-18624_set99_feat172_valid-pm,CV5-49258_set99_feat172_valid-pm,CV6-80683_set99_feat172_valid-pmo,CV3-65171_set99_feat152_valid-0,CV3-73848_set99_feat172_valid-pmo,CV4-77547_set99_feat172_valid-pm,CV3-86836_set99_feat172_valid-pm,CV4-38035_set99_feat172_valid-pm,CV5-80228_set99_feat172_valid-pm,CV4-00942_set99_feat172_valid-pm,CV3-40511_set99_feat172_valid-pmo,CV3-71941_set99_feat172_valid-pmo,CV4-35107_set99_feat172_valid-pmo,CV3-64234_set99_feat180_valid-0,CV3-64976_set99_feat170_valid-0,CV3-64974_set99_feat175_valid-0,CV3-65038_set99_feat171_valid-0,CV4-08712_set99_feat172_valid-pmo,CV3-37315_set99_feat172_valid-pmo,CV3-35679_set99_feat172_valid-pmo,CV6-71242_set99_feat172_valid-pm,CV6-76957_set99_feat172_valid-pmo,CV3-64913_set99_feat200_valid-0,CV3-27036_set99_featstack

In [21]:
train = base[~base[target].isnull()]
# train[train.isnull()]
for col in train.columns:
    if train[col].isnull().sum()>0:
        print(col)

CV3-35304_set99_feat188_valid-0
CV3-35034_set99_feat200_valid-0
CV3-35023_set99_feat204_valid-0
